### Advan Research Exploration for Walmarts

In [2]:
import sys
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import re
import pandas as pd

sys.path.append(str(Path.cwd().parent))
from config import PATH_KIOSK_USER_PATTERNS_FOLDER, PATH_KIOSK_USER_PATTERNS_REPO


kupdat03 = ds.dataset(PATH_KIOSK_USER_PATTERNS_FOLDER / "working/processed/kupdat03_advan research monthly patterns", 
    format="parquet", partitioning=["YEAR", "MONTH"])


In [7]:
kupdat03.head(10).to_pandas()

,CITY,DATE_RANGE_END,DATE_RANGE_START,LATITUDE,LOCATION_NAME,LONGITUDE,NAICS_CODE,PLACEKEY,RAW_VISITOR_COUNTS,RAW_VISIT_COUNTS,REGION,STREET_ADDRESS,FILE_NAME,HOME_CBG,VISITOR_COUNT,YEAR,MONTH
0,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150003001,110,YEAR=2019,MONTH=1
1,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150009002,73,YEAR=2019,MONTH=1
2,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150021002,60,YEAR=2019,MONTH=1
3,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150005021,56,YEAR=2019,MONTH=1
4,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150005022,55,YEAR=2019,MONTH=1
5,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150005023,55,YEAR=2019,MONTH=1
6,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150009004,52,YEAR=2019,MONTH=1
7,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150017005,45,YEAR=2019,MONTH=1
8,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150020021,43,YEAR=2019,MONTH=1
9,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150004005,42,YEAR=2019,MONTH=1


## Unique LOCATION_NAME and counts in kupdat03


In [ ]:
# Get unique LOCATION_NAME values and counts of those values without loading the entire dataset
# Use PyArrow to compute counts efficiently
location_counts = kupdat03.scanner(columns=["LOCATION_NAME"]).to_table().group_by("LOCATION_NAME").aggregate([("LOCATION_NAME", "count")]).to_pandas()
location_counts.columns = ["LOCATION_NAME", "COUNT"]
location_counts = location_counts.sort_values("COUNT", ascending=False)

# Display the top results
print(f"Total unique locations: {len(location_counts)}")


location_counts.to_csv(PATH_KIOSK_USER_PATTERNS_REPO/"analysis/kupana03_walmart location names and counts.csv", index=False)

Total unique locations: 2936


In [ ]:
location_counts.head(20)



,LOCATION_NAME,COUNT
0,Walmart Pharmacy,34717022
1,Walmart Vision Center,31062566
8,Walmart Photo Center,30712579
10,Walmart Auto Care Center,24814232
4,Walmart Supercenter,18373377
2,Walmart Fuel Station,2280110
6,Walmart Neighborhood Market,1824480
5,Walmart,1596187
7,Walmart Distribution,830694
206,Sunbelt Rentals Inside Walmart,300644


In [14]:

# Select the locations to keep:
selected_locations = ["Walmart Pharmacy","Walmart Supercenter","Walmart","Walmart Center","Walmart Neighborhood Market"]
# Create a regex pattern to match both "Walmart" and "Supercenter" in any order
pattern = re.compile(r'(?=.*Walmart)(?=.*Supercenter)', re.IGNORECASE)

# Filter locations that are in our selected list or match the pattern
def filter_location(loc):
    if loc in selected_locations:
        return True
    # Check if it matches "Walmart" and "Supercenter" in any order
    elif pattern.search(loc):
        return True
    return False

# Get all unique location names from our location_counts dataframe
all_locations = location_counts['LOCATION_NAME'].tolist()

# Apply our filter to get the list of locations to keep
locations_to_keep = [loc for loc in all_locations if filter_location(loc)]

# Print the number of locations that will be kept
print(f"Keeping {len(locations_to_keep)} out of {len(all_locations)} locations")

# Display the first few locations we're keeping
print("Sample locations being kept:")
for loc in locations_to_keep[:10]:
    print(f"  - {loc}")

# Create a filtered dataset with only the selected locations
filtered_kupdat03 = kupdat03.filter(ds.field('LOCATION_NAME').isin(locations_to_keep))

Keeping 2343 out of 2936 locations
Sample locations being kept:
  - Walmart Pharmacy
  - Walmart Supercenter
  - Walmart Neighborhood Market
  - Walmart
  - Walmart Center
  - Walmart Supercenter San Antonio Supercenter
  - Walmart Supercenter Houston Supercenter
  - Walmart Supercenter Jacksonville Supercenter
  - Walmart Supercenter Dallas Supercenter
  - Walmart Supercenter Las Vegas Supercenter


In [15]:
# Count total rows in filtered_kupdat03
row_count = filtered_kupdat03.scanner().count_rows()
print(f"Total rows in filtered_kupdat03: {row_count:,}")

# Get a sample of the data to verify contents
sample_df = filtered_kupdat03.head(5).to_pandas()
print("\nSample data:")
display(sample_df)



Total rows in filtered_kupdat03: 69,277,648

Sample data:


,CITY,DATE_RANGE_END,DATE_RANGE_START,LATITUDE,LOCATION_NAME,LONGITUDE,NAICS_CODE,PLACEKEY,RAW_VISITOR_COUNTS,RAW_VISIT_COUNTS,REGION,STREET_ADDRESS,FILE_NAME,HOME_CBG,VISITOR_COUNT,YEAR,MONTH
0,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150003001,110,YEAR=2019,MONTH=1
1,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150009002,73,YEAR=2019,MONTH=1
2,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150021002,60,YEAR=2019,MONTH=1
3,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150005021,56,YEAR=2019,MONTH=1
4,Longview,2019-02-01 00:00:00.000,2019-01-01,46.119351,Walmart Pharmacy,-122.930115,446110,222-223@5xd-j6x-p5f,3220.0,6827.0,WA,540 7th Ave,data_01bd7440-0105-dcc9-0042-fa0702ed2712_03_1...,530150005022,55,YEAR=2019,MONTH=1


In [16]:
# Count rows by location type to see distribution
location_distribution = filtered_kupdat03.scanner(columns=["LOCATION_NAME"]).to_table().group_by("LOCATION_NAME").aggregate([("LOCATION_NAME", "count")]).to_pandas()
location_distribution.columns = ["LOCATION_NAME", "COUNT"]
location_distribution = location_distribution.sort_values("COUNT", ascending=False)
display(location_distribution.head(10))

,LOCATION_NAME,COUNT
0,Walmart Pharmacy,34717022
1,Walmart Supercenter,18373377
3,Walmart Neighborhood Market,1824480
2,Walmart,1596187
4,Walmart Center,150905
308,Walmart Supercenter San Antonio Supercenter,125979
671,Walmart Supercenter Houston Supercenter,117977
51,Walmart Supercenter Jacksonville Supercenter,83993
198,Walmart Supercenter Dallas Supercenter,71571
212,Walmart Supercenter Las Vegas Supercenter,54287
